<a href="https://colab.research.google.com/github/Allzer/Neural-networks-on-Python/blob/main/10_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import re

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, SimpleRNN, Input, Dropout, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence

In [22]:
with open('Пасты_2.txt', 'r', encoding='utf-8') as f:
  text = f.read()

Пропускаем текст через токенайзер с задаными параметрами редактивроания текста

Смотрим сколько раз встречается каждое слово

In [23]:
maxWordsCount = 3875
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                      lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([text])
dist = list(tokenizer.word_counts.items())
print(dist[:10])

data = tokenizer.texts_to_sequences([text]) #преоразуем текст в последовательность чисел в соответствии с полученным словарём (берём каждое слово в нашем тексте и вместо него ставим число, которое ему соответствует)
#res = to_categorical(data[0], num_classes=maxWordsCount)
#print(res.shape)

res = np.array(data[0])

inp_words = 3
n = res.shape[0] - inp_words

x = np.array([res[i:i + inp_words] for i in range(n)])
y = to_categorical(res[inp_words:], num_classes=maxWordsCount)

[('мы', 284), ('с', 744), ('пацанами', 6), ('живем', 1), ('в', 1923), ('славном', 1), ('городе', 3), ('якутск', 1), ('это', 408), ('такое', 31)]


In [34]:
model = Sequential()

model.add(Embedding(maxWordsCount, 256, input_length = inp_words))
model.add(SimpleRNN(300, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [35]:
history = model.fit(x,y, batch_size=32, epochs=50)

Epoch 1/50
1651/1651 [==============================] - 65s 39ms/step - loss: 6.7315 - accuracy: 0.0574
Epoch 2/50
1651/1651 [==============================] - 66s 40ms/step - loss: 5.9794 - accuracy: 0.0979
Epoch 3/50
1651/1651 [==============================] - 66s 40ms/step - loss: 4.9358 - accuracy: 0.1664
Epoch 4/50
1651/1651 [==============================] - 64s 39ms/step - loss: 3.8548 - accuracy: 0.2647
Epoch 5/50
1651/1651 [==============================] - 65s 39ms/step - loss: 2.9968 - accuracy: 0.3836
Epoch 6/50
1651/1651 [==============================] - 64s 39ms/step - loss: 2.3642 - accuracy: 0.4889
Epoch 7/50
1651/1651 [==============================] - 65s 39ms/step - loss: 1.9000 - accuracy: 0.5783
Epoch 8/50
1651/1651 [==============================] - 64s 39ms/step - loss: 1.5625 - accuracy: 0.6484
Epoch 9/50
1651/1651 [==============================] - 64s 39ms/step - loss: 1.3090 - accuracy: 0.7040
Epoch 10/50
1651/1651 [==============================] - 60s 36m

In [46]:
def buildPhrase(texts, str_len=30):
    res = texts
    data = tokenizer.texts_to_sequences([texts])[0]
    for i in range(str_len):
        #x = to_categorical(data[i: i + inp_words], num_classes=maxWordsCount)  # преобразуем в One-Hot-encoding
        #inp = x.reshape(1, inp_words, maxWordsCount)

        x = data[i: i + inp_words]
        inp = np.expand_dims(x, axis=0)

        pred = model.predict(inp)
        indx = pred.argmax(axis=1)[0]
        data.append(indx)

        res += " " + tokenizer.index_word[indx]  # дописываем строку

    return res

In [48]:
res = buildPhrase("был дома и снимал порно")
print(res)

1/1 [==============================] - 0s 21ms/step
был дома и снимал порно этим и вдруг я почувствовал какое то урчание в животе моего кишечника я ощутил чего то мерзкого в своем кишечнике и тут струя поноса что есть силы сжал очко и
